<a href="https://colab.research.google.com/github/Dan1lk/data_generator/blob/main/data_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=5b44acb9dbfb48585c678bd31073c54e7eaf8d0e270fc4a36706d05adb8c8e20
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [15]:
from pyspark.sql import SparkSession
from random import choice, randint, random
import json
from datetime import datetime, timedelta
from math import floor
import os


spark = SparkSession.builder \
    .appName("Generate synthetic data") \
    .config('spark.master', 'local[*]') \
    .config('spark.driver.memory', '4g') \
    .config('spark.executor.cores', '4') \
    .config('spark.executor.memory', '8g') \
    .getOrCreate()



email_example = ['yandex.ru', "mail.ru", "gmail.ru", "inbox.ru", "ok.ru", "rambler.ru", "yahoo.com", "usa.com", "citeweb.com", "deneg.com", "england.com", "hotmail.com", "mail.com", "jetterbox.com"]
salary_example = [i for i in range(100000, 500000, 10000)]


#Заполняем лист именами из файла
names = []
with open("russian_trans_names.txt", "r", encoding="utf-8") as file:
    for i in file.readlines():
        i_strip = i.strip("\n")
        if len(i_strip) >= 5:
            names.append(i.strip("\n").capitalize())

#Заполняем лист городами из файла
cities = []
with open("russian-cities.json", 'r', encoding='utf-8') as file:
    cities_json = json.load(file)
    for i in cities_json:
        if len(i["name"]) >= 7:
            cities.append(i["name"])

#Заполняем лист email
email = []
for i in names:
    email.append(f"{i.lower()}@{choice(email_example)}")



#функция выбирает рандомную дату регистрации, учитывая что дата регистрации не может быть старше возраста челловека
str_count = int(input('Введите количество строк: '))

def random_data_registry(age):
    years_ago = floor(random() * age)
    data_now = datetime.now().date()
    data_registry = data_now - timedelta(weeks=years_ago * 52) - timedelta(days=years_ago * 1)
    return data_registry.strftime("%Y-%m-%d")

#функция возвращает true, если random возвращает число > 0.05 (в 5% случаев), а иначе None
def random_num():
    if random() > 0.05:
        return True

#Заполняем лист синтетическими данными на str_count элементов
data = []
for i in range(1, str_count + 1):

    ch_name = choice(names)
    ch_city = choice(cities)
    age = randint(18, 95)

    str_data = (i, ch_name if random_num() else None, email[names.index(ch_name)] if random_num() else None, \
                ch_city if random_num() else None, age if random_num() else None, choice(salary_example) if random_num() else None, \
                random_data_registry(age) if random_num() else None)
    data.append(str_data)
#создаем датафрейм
df = spark.createDataFrame(data, ["id", "name", "email", "city", "age", "salary", "registration_date"])
#путь сохранения файла csv
path = f"{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}-dev.csv"
#обьединяем партиции в один файл и записываем на диск
df.coalesce(1).write.csv(path, header=True, mode='overwrite')


#переименование файла

#находим название старого файла
old_file_name = [i for i in os.listdir(path) if i.endswith(".csv")][0]
#путь к файлу со старым названием
old_file_path = os.path.join(path, old_file_name)
#имя для нового файла
new_file_name = path
#путь для нового файла
new_file_path = os.path.join(path, new_file_name)
#меняем название файла на новое
os.rename(old_file_path, new_file_path)

#путь к success файлу
success_file_path = os.path.join(path, '_SUCCESS')
#удаляем этот файл из папки
os.remove(success_file_path)


spark.stop()

Введите количество строк: 100
